<a href="https://colab.research.google.com/github/jmlDC/Truth-Thesis22-23/blob/Scraping-trial/ScrapingNews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
dir  = "/content/gdrive/MyDrive/THESIS-MS/Truth-Thesis22-23/"


In [4]:

# filename= f"{dir}NEWS.csv"
# f=open(filename,"w", encoding = 'utf-8')
# headers="Statement,Link,Date, Source, Label\n"
# f.write(headers)
# f.close()

In [5]:
## Politifact 
pagesToGet= 1
upperframe=[]  
site = "politifact"


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.

    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))

    
    filename= f"{dir + site}NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date, Source, Label\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'m-statement__body'}).find('footer').text[-14:-1].strip()
        Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date,Source,Label))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+","+Source.replace(",","^")+","+Label.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date','Source','Label'])
data.head()

processing page : 1
https://www.politifact.com/factchecks/list/?page=1
30


,Statement,Link,Date,Source,Label
0,Video shows Joe Biden has a body double.,https://www.politifact.com/factchecks/2022/sep...,"ember 7, 2022",Viral image,pants-fire
1,“The U.S. & Elon Musk reveal new hypersonic je...,https://www.politifact.com/factchecks/2022/sep...,"ember 7, 2022",Bloggers,false
2,The National Institutes of Health “adds iverme...,https://www.politifact.com/factchecks/2022/sep...,"ember 7, 2022",Instagram posts,false
3,The child tax credit passed under President Jo...,https://www.politifact.com/factchecks/2022/sep...,"ember 7, 2022",Raphael Warnock,half-true
4,There’s an unlimited supply of water being gen...,https://www.politifact.com/factchecks/2022/sep...,"ember 7, 2022",Instagram posts,barely-true


In [6]:
def extract_source(url):
    agent = {"User-Agent":"Mozilla/5.0"}
    try:
      #use the browser to get the url. This is suspicious command that might blow up.
      # page=requests.get(url)                             # this might throw an exception if something goes wrong.
      source=requests.get(url, headers=agent)
      

    except Exception as e:                                   # this describes what to do if an exception is thrown
      error_type, error_obj, error_info = sys.exc_info()      # get the exception information
      print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
      print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
    
    return source                                               #ignore this page. Abandon this and go back.

    

# def extract_data(source):
#      soup=BeautifulSoup(source, 'lxml')
#      names=soup.findAll('title')
#      for i in names:
#        print(i)  

In [7]:
# extract_data(extract_source(url).text)


NameError: ignored

In [26]:
## Manila bulletin
pagesToGet= 1
upperframe=[]  
topic = "ukraine war"
site = "manilaBulletin"


for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://mb.com.ph/page/'+str(page)+'/?s='+topic.replace(" ", "+")
    print(url)

    time.sleep(2)   
    # soup=BeautifulSoup(page.text,'html.parser')
    soup=BeautifulSoup(extract_source(url).text, 'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'article'})
    print(len(links))
    
    
    filename= f"{dir + site}NEWS.csv"
    f=open(filename,"w", encoding = 'utf-8')
    headers="Statement,Link,Date\n"
    f.write(headers)
    
    for j in links:
        Statement = j.find("h4",attrs={'class':'title'}).text.strip() 
        # Link = "https://www.mb.com.ph/"
        Link = j.find("h4",attrs={'class':'title'}).find('a')['href'].strip()
        Date = j.find('div',attrs={'class':'post-meta'}).find('span', attrs={'class':'time-posted'}).find('span',attrs={'class': 'count'}).dtype()  #.text[:9].strip()
        # Source = j.find('div', attrs={'class':'m-statement__meta'}).find('a').text.strip()
        # Label = j.find('div', attrs ={'class':'m-statement__content'}).find('img',attrs={'class':'c-image__original'}).get('alt').strip()
        frame.append((Statement,Link,Date))
        f.write(Statement.replace(",","^")+","+Link+","+Date.replace(",","^")+"\n")
    upperframe.extend(frame)
f.close()
data=pd.DataFrame(upperframe, columns=['Statement','Link','Date'])
data.head()

processing page : 1
https://mb.com.ph/page/1/?s=ukraine+war
41


TypeError: ignored

In [ ]:
print(soup)

In [ ]:
!ls
# !rm {dir\}NEWS-test.csv